In [0]:
%pip install mlflow openai databricks-agents
dbutils.library.restartPython()

In [0]:
%run ./setup

## Nota Bene
Bug in mlflow related to git folders. You must explicitly set the experiment if you want to create a labelling session

In [0]:
import mlflow
mlflow.set_experiment("/Users/ben.mackenzie@databricks.com/agent_eval_chatbot")

In [0]:
import mlflow
from openai import OpenAI

# Enable automatic tracing for all OpenAI API calls
#mlflow.openai.autolog()

# Connect to a Databricks LLM via OpenAI using the same credentials as MLflow
# Alternatively, you can use your own OpenAI credentials here
mlflow_creds = mlflow.utils.databricks_utils.get_databricks_host_creds()
client = OpenAI(
    api_key=mlflow_creds.token,
    base_url=f"{mlflow_creds.host}/serving-endpoints"
)

# Create a RAG app with tracing
@mlflow.trace
def my_chatbot(user_question: str) -> str:
    # Retrieve relevant context
    context = retrieve_context(user_question)

    # Generate response using LLM with retrieved context
    response = client.chat.completions.create(
        model="databricks-llama-4-maverick",  # If using OpenAI directly, use "gpt-4o" or "gpt-3.5-turbo"
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Use the provided context to answer questions."},
            {"role": "user", "content": f"Context: {context}\n\nQuestion: {user_question}"}
        ],
        temperature=0.7,
        max_tokens=150
    )
    return response.choices[0].message.content

@mlflow.trace(span_type="RETRIEVER")
def retrieve_context(query: str) -> str:
    # Simulated retrieval - in production, this would search a vector database
    if "mlflow" in query.lower():
        return "MLflow is an open-source platform for managing the end-to-end machine learning lifecycle. It provides tools for experiment tracking, model packaging, and deployment."
    return "General information about machine learning and data science."



In [0]:
# Run the app to generate a trace
response = my_chatbot("What is MLflow?")
print(f"Response: {response}")

# Get the trace ID for the next step
trace_id = mlflow.get_last_active_trace_id()
print(f"Trace ID: {trace_id}")

In [0]:
import mlflow
from mlflow.entities.assessment import AssessmentSource, AssessmentSourceType

# Simulate end-user feedback from your app
# In production, this would be triggered when a user clicks thumbs down in your UI
mlflow.log_feedback(
    trace_id=trace_id,
    name="user_feedback",
    value=False,  # False for thumbs down - user is unsatisfied
    rationale="Missing details about MLflow's key features like model evaluation",
    source=AssessmentSource(
        source_type=AssessmentSourceType.HUMAN,
        source_id="enduser_123",  # Would be actual user ID in production
    ),
)

print("✅ End-user feedback recorded!")

# In a real app, you would:
# 1. Return the trace_id with your response to the frontend
# 2. When user clicks thumbs up/down, call your backend API
# 3. Your backend would then call mlflow.log_feedback() with the trace_id

## Nota Bene
End user feeback is an assessment (could be used by a custom Scorer but not used by e.g., Correctness scorer which requires expected facts or expected response)

In [0]:
import mlflow
from mlflow.genai.label_schemas import create_label_schema, InputCategorical, InputText, InputTextList


# Define what feedback to collect
accuracy_schema = create_label_schema(
    name="response_accuracy",
    type="feedback",
    title="Is the response factually accurate?",
    input=InputCategorical(options=["Accurate", "Partially Accurate", "Inaccurate"]),
    overwrite=True
)

expected_facts_schema = create_label_schema(
    name="expected_facts",
    type="expectation",
    title="What are the expected facts?",
    input=InputTextList(),
    overwrite=True
)



In [0]:
from mlflow.genai.labeling import create_labeling_session

# Create a labeling session
labeling_session = create_labeling_session(
    name="review_with_expected_facts_2",
    label_schemas=["expected_facts", "response_accuracy"],
)
# Add your trace to the session
# Get the most recent trace from the current experiment
traces = mlflow.search_traces(
    max_results=1  # Gets the most recent trace
)
labeling_session.add_traces(traces)

# Share with reviewers
print(f"✅ Trace sent for review!")
print(f"Share this link with reviewers: {labeling_session.url}")

In [0]:
labeled_traces = mlflow.search_traces(
    run_id=labeling_session.mlflow_run_id,  # Labeling Sessions are MLflow Runs
)

# or 
# labeled_traces = mlflow.search_traces(run_id="5387ba7c2a284e0d92ebf5e3a2c6b1ab")

Have a look at trace in experiment

In [0]:
import mlflow
import mlflow.genai.datasets
import time


# 1. Create an evaluation dataset

evaluation_dataset_table_name = "human_feedback_eval_2"

UC_TABLE_NAME = f"{eval_schema}.{evaluation_dataset_table_name}"

eval_dataset = mlflow.genai.datasets.create_dataset(
    uc_table_name=UC_TABLE_NAME,
)

#eval_dataset = mlflow.genai.datasets.get_dataset(UC_TABLE_NAME)
print(f"Created evaluation dataset: UC_TABLE_NAME")

eval_dataset = eval_dataset.merge_records(labeled_traces)
print(f"Added {len(traces)} records to evaluation dataset")



## Nota Bene
We are creating an eval data set using traces.  We could have created with a list...but we would have had to add expectations to run Correctness Scorer

In [0]:
import mlflow
from mlflow.genai.scorers import Correctness

# Evaluate your app against expert expectations
with mlflow.start_run(run_name="v1"):
    eval_results = mlflow.genai.evaluate(
        data=eval_dataset,
        predict_fn=my_chatbot,  # The app we created in Step 1
        scorers=[Correctness()]  # Compares outputs to expected_response
    )

##Nota Bene

Running eval on a dataset created from traces creates new traces. The new trace does not contain the original human assesemment but does include the expectations created in labelling session.

Lets try to create a labelling session through the UI.  First lets' create a new trace.

In [0]:
response = my_chatbot("What is the difference between databricks MLflow? and open source mlflow?")
print(f"Response: {response}")

# Get the trace ID for the next step
trace_id = mlflow.get_last_active_trace_id()
print(f"Trace ID: {trace_id}")

Now add the new trace with assessment to the evaluation dataset.




##Nota Bene
This can be done through UI..but there is a bug the create multiple entries in selection box.

In [0]:
#not sure how how to find a trace based on the id...

traces = mlflow.search_traces(
    filter_string="tags.demo = '1'"
)
display(traces)

In [0]:
eval_dataset = eval_dataset.merge_records(traces)

## Go to UI to create Labelling Session.  Then come back and run evaluation


In [0]:
import mlflow
from mlflow.genai.scorers import Correctness

# Evaluate your app against expert expectations
with mlflow.start_run(run_name="v4"):
    eval_results = mlflow.genai.evaluate(
        data=eval_dataset,
        predict_fn=my_chatbot,  # The app we created in Step 1
        scorers=[Correctness()]  # Compares outputs to expected_response
    )